**환경 설정**

In [ ]:
!pip install ipython-sql==0.4.1
!pip install SQLAlchemy==1.4.49

In [3]:
%load_ext sql

## JOIN:

An SQL JOIN clause is used to combine rows from two or more tables, based on a common field between them.

왼쪽 테이블을 LEFT라고 하고 오른쪽 테이블을 RIGHT이라고 하자. JOIN의 결과는 방식에 상관없이 양쪽의 필드를 모두 가진 새로운 테이블을 만들어내게 됨. 조인의 방식에 따라 다음 두 가지가 달라짐:

1. 어떤 레코드들이 선택되는지?
2. 어떤 필드들이 채워지는지?

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.vital;
CREATE TABLE raw_data.vital (
    UserID int,
    VitalID	int,
    Date date,
    Weight int
)
;
INSERT INTO raw_data.vital VALUES
(100,	1, '2020-01-01', 75),
(100, 3, '2020-01-02', 78),
(101, 2, '2020-01-01', 90),
(101, 4, '2020-01-02', 95);

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.alert;
CREATE TABLE raw_data.alert (
    AlertID int,
    VitalID	int,
    AlertType varchar(32),
    Date date,
    UserID int
);
INSERT INTO raw_data.alert VALUES
 (1,	4, 'WeightIncrease', '2020-01-01', 101),
 (2, NULL, 'MissingVital', '2020-01-04', 100),
 (3, NULL, 'MissingVital', '2020-01-04', 101);

* INNER JOIN
1. 양쪽 테이블에서 매치가 되는 레코드들만 리턴함
2. 양쪽 테이블의 필드가 모두 채워진 상태로 리턴됨

In [ ]:
%%sql

SELECT * FROM raw_data.vital v
JOIN raw_data.alert a ON v.vitalid = a.vitalid;


* LEFT JOIN: 왼쪽 테이블의 레코드는 모두 리턴되며 오른쪽 테이블과 매칭이 되는 레코드들의 경우에는 오른쪽 테이블 레코드들의 컬럼들이 채워진 상태로 리턴됨. 매칭이 안되는 왼쪽 테이블 레코드들은 오른쪽 테이블에서 들어오는 필드들은 NULL로 채워짐

In [ ]:
%%sql

SELECT * FROM raw_data.vital v
LEFT JOIN raw_data.alert a ON v.vitalid = a.vitalid;

* FULL JOIN (OUTER JOIN):
1. 왼쪽 테이블과 오른쪽 테이블의 모든 레코드들을 리턴함
2. 매칭되는 경우에만 양쪽 테이블들의 모든 필드들이 채워진 상태로 리턴됨

In [ ]:
%%sql

SELECT * FROM raw_data.vital v
FULL JOIN raw_data.alert a ON v.vitalid = a.vitalid;

* CARTESIAN JOIN (CROSS JOIN): 조인 조건 없이 두 개 테이블의 내용을 모두 조합한 결과 레코드들을 생성

In [ ]:
%%sql

SELECT *
FROM (
    SELECT vitalid  -- 1,2,3,4
    FROM raw_data.vital
) v
CROSS JOIN (
    SELECT alertid   -- 1,2,3
    FROM raw_data.alert
) a;

In [ ]:
%%sql

SELECT * FROM raw_data.vital v
CROSS JOIN raw_data.alert a;

SELF JOIN

In [ ]:
%%sql

SELECT * FROM raw_data.vital v1
JOIN raw_data.vital v2 ON v1.vitalid = v2.vitalid;

## Boolean과 COALESCE/NULLIF

In [ ]:
%%sql

SELECT *
FROM raw_data.boolean_test;

In [ ]:
%%sql

SELECT
    COUNT(CASE WHEN flag = True THEN 1 END) true_cnt1,
    COUNT(CASE WHEN flag is True THEN 1 END) true_cnt2,
    COUNT(CASE WHEN flag is not False THEN 1 END) not_false_cnt
FROM raw_data.boolean_test;

In [ ]:
%%sql

SELECT COUNT(1)
FROM raw_data.boolean_test
WHERE flag is NULL;

In [ ]:
%%sql

SELECT COUNT(1)
FROM raw_data.boolean_test
WHERE flag = NULL;

공백이 들어있는 필드이름 혹은 예약된 키워드를 필드이름으로 사용하려면?

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.keeyong_test;
CREATE TABLE adhoc.keeyong_test (
    group int primary key,
    'mailing address' varchar(32)
);

In [ ]:
%%sql

CREATE TABLE adhoc.keeyong_test (
    "group" int primary key,
    "mailing address" varchar(32)
);

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.keeyong_test_null;
CREATE TABLE adhoc.keeyong_test_null (
    value int NOT NULL
);

In [ ]:
%%sql

INSERT INTO adhoc.keeyong_test_null VALUES (NULL);

# **숙제 - 1**

**채널별 월 매출액 테이블 만들기 (adhoc 스키마 밑에 CTAS로 본인 영문 이름을 포함해서 테이블 만들기)**
1.   session_timestamp, user_session_channel, channel, session_transaction 테이블들을 사용

2.   아래와 같은 필드로 구성
    - month
    - channel
    - uniqueUsers (총방문 사용자)
    - paidUsers (구매 사용자: refund한 경우도 판매로 고려)
    - conversionRate (구매사용자 / 총방문 사용자)
    - grossRevenue (refund 포함)
    - netRevenue (refund 제외)




**1. 혹시 OUT JOIN이 필요한지, 테이별 점검 필요**

In [ ]:
%%sql

select distinct sessionid from raw_data.session_timestamp
minus
select distinct sessionid from raw_data.user_session_channel
;

In [ ]:
%%sql

select distinct sessionid from raw_data.user_session_channel
minus
select distinct sessionid from raw_data.session_timestamp
;

In [ ]:
%%sql

select * from raw_data.session_transaction
where amount <= 0

**2. Summary Table 만들기**

In [ ]:
%%sql

SELECT LEFT(ts, 7) "month",  -- "year month"
       channel,
       COUNT(DISTINCT usc.userid) uniqueUsers,
       COUNT(DISTINCT CASE WHEN amount > 0 THEN usc.userid END) paidUsers,
       ROUND(paidUsers*100.0/NULLIF(uniqueUsers, 0),2) conversionRate,
       SUM(amount) grossRevenue,
       SUM(CASE WHEN refunded is False THEN amount END) netRevenue
   FROM raw_data.user_session_channel usc
   LEFT JOIN raw_data.session_timestamp t ON t.sessionid = usc.sessionid
   LEFT JOIN raw_data.session_transaction st ON st.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;

In [ ]:
%%sql

--혹시 기존에 생성되어 있으면 삭제
DROP TABLE IF EXISTS adhoc.keeyong_monthly_channel_summary;

--Summary Table 생성
CREATE TABLE adhoc.keeyong_monthly_channel_summary
AS
SELECT TO_CHAR(ts, 'YYYY-MM') year_month,
    usc.channel,
    COUNT(DISTINCT usc.userid) unique_users,
    COUNT(DISTINCT CASE WHEN amount>0 THEN userid END) paid_users,
    ROUND(paid_users*100./NULLIF(unique_users,0),2) conversion_rate,
    SUM(amount) gross_revenue,
    SUM(CASE WHEN refunded is False THEN amount
        ELSE 0 END) net_revenue
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
  LEFT JOIN raw_data.session_transaction str ON usc.sessionid = str.sessionid
GROUP BY 1, 2;

--정상적으로 생성되었는지 확인
SELECT * FROM adhoc.keeyong_monthly_channel_summary;


In [ ]:
%%sql

SELECT *
FROM raw_data.count_test;

In [ ]:
%%sql

SELECT
     value,
     COALESCE(value, 0)
FROM raw_data.count_test;

숙제1: 사용자별로 처음 채널과 마지막 채널 알아내기
- ROW_NUMBER vs FIRST_VALUE/LAST_VALUE
- 사용자 251번의 시간순으로 봤을 떄 첫 번째 채널과 마지막 채널은 무엇인가?
  + 노가다를 하자면 아래 쿼리를 실행해서 처음과 마지막 채널을 보면 됨
```sql
SELECT ts, channel
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
WHERE userid = 251
ORDER BY 1
```
- ROW_NUMBER를 이용해서 해보자
  + ROW_NUMBER() OVER(PARTITION BY field1 ORDER BY field2)nn

In [ ]:
%%sql

SELECT
  COUNT(DISTINCT userid)
FROM raw_data.user_session_channel;

In [ ]:
%%sql

WITH RN AS (
SELECT
  userid,
  ts,
  channel,
  ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) num1,
  ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) num2
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)

SELECT
  userid,
  MAX(CASE WHEN num1 = 1 THEN channel END) first_chanel,
  MAX(CASE WHEN num2 = 1 THEN channel END) last_chanel
FROM RN
GROUP BY userid
ORDER BY 1

숙제2: Gross RevenueRK 가장 큰 UserID 10개 찾기
- user_session_channel과 session_transaction과 session_timestamp 테이블을 이용
- Gross revenue: Refund 포함한 매출

In [ ]:
%%sql

SELECT COUNT(*)
FROM raw_data.session_timestamp;

In [ ]:
%%sql

SELECT COUNT(*)
FROM raw_data.user_session_channel;

In [ ]:
%%sql

SELECT
  usc.userid,
  SUM(st.amount) gross_revenue
FROM raw_data.user_session_channel usc
JOIN raw_data.session_transaction st ON usc.sessionid = st.sessionid
GROUP BY usc.userid
ORDER BY gross_revenue DESC
LIMIT 10;



숙제3: raw_data.nps 테이블을바탕으로 월별 NPS 계산
- 고객들이 0(의향 없음)에서 10(의향 아주 높음)
- detractor(비추전자): 0 에서 6
- passive(소극자): 7 에서 8
- promoter(홍보자): 9 나 10
- NPS = promoter 퍼센트 - detractor 퍼센트

In [ ]:
%%sql

WITH nps_trans AS(
SELECT
  TO_CHAR(created_at,'YYYY-MM') year_month,
  CASE WHEN score > 8 THEN 'promoter'
  WHEN score < 7 THEN 'detractor'
  ELSE 'passive' END AS survey
FROM raw_data.nps)

SELECT
  year_month,
  count(year_month),
  SUM(CASE WHEN survey = 'promoter' THEN 1 ELSE 0 END)*100.0/COUNT(year_month) AS promoter,
  SUM(CASE WHEN survey = 'detractor' THEN 1 ELSE 0 END)*100.0/COUNT(year_month) AS detractor,
  SUM(CASE WHEN survey = 'passive' THEN 1 ELSE 0 END)*100.0/COUNT(year_month) AS passive,
  ROUND(promoter - detractor,2) AS nps
FROM nps_trans
GROUP BY year_month
ORDER BY year_month
